# TP2 - Organización de Datos
#### Notebook principal

<hr>

### Notebooks utilizados:

- ***pre_processing:*** notebook para el manejo inicial de los dataframes.
- ***feature_generation:*** primer etapa del pipeline. En este notebook se generarán nuevos features para luego, realizar un proceso de selección de los mejores features para cada modelo.
- ***feature_selection*** segunda etapa, donde se buscara encontrar los features con mayor importancia, es decir aquellos que aporten mayor informacion.
- ***parameter_tuning:*** tercer etapa, notebook donde se tunean los parámetros para cada modelo.
- ***predict:*** finalmente, una vez obtenidos los mejores parametros y features para cada modelo, este notebook se encargará de generar el csv con las predicciones finales para el modelo que se le indique.

<hr>


In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

seed = 7

In [2]:
import nbimporter

import pre_processing
import feature_generation
import feature_selection
import parameter_tuning
import predict

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_generation.ipynb
Importing Jupyter notebook from feature_selection.ipynb
Importing Jupyter notebook from parameter_tuning.ipynb
Importing Jupyter notebook from predict.ipynb


In [3]:
def escribir_respuesta(ids,predicciones):
    with open("respuesta.csv",'w') as archivo:
        archivo.write("id,target\n")
        for i in range(len(ids)):
            linea = f"{int(ids[i])},{predicciones[i]}"
            archivo.write(f"{linea}\n")

<hr>

# Resultados obtenidos

# area de testing:

In [6]:
import lightgbm as lgb

In [7]:
train,test = pre_processing.load_featured_datasets()

In [8]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [9]:
features = feature_generation.get_features()

In [10]:
train_selected = feature_selection.get_selected_dataframe(train)
test_selected = feature_selection.get_selected_dataframe(test, precio=False)

In [11]:
X = train_selected.drop('precio', axis=1)
Y = train_selected['precio']

In [12]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.05, random_state=seed)

In [13]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 7,
    'num_leaves': 80,
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 200,
    'n_jobs':2}
n_estimators=20000

In [14]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 0.660884
Training until validation scores don't improve for 200 rounds
[2]	valid_0's l1: 0.635049
[3]	valid_0's l1: 0.610543
[4]	valid_0's l1: 0.58776
[5]	valid_0's l1: 0.566114
[6]	valid_0's l1: 0.545677
[7]	valid_0's l1: 0.526149
[8]	valid_0's l1: 0.50793
[9]	valid_0's l1: 0.49073
[10]	valid_0's l1: 0.474442
[11]	valid_0's l1: 0.459304
[12]	valid_0's l1: 0.445103
[13]	valid_0's l1: 0.431449
[14]	valid_0's l1: 0.418925
[15]	valid_0's l1: 0.406892
[16]	valid_0's l1: 0.39573
[17]	valid_0's l1: 0.385114
[18]	valid_0's l1: 0.375394
[19]	valid_0's l1: 0.366041
[20]	valid_0's l1: 0.357209
[21]	valid_0's l1: 0.349104
[22]	valid_0's l1: 0.341337
[23]	valid_0's l1: 0.33408
[24]	valid_0's l1: 0.327302
[25]	valid_0's l1: 0.32105
[26]	valid_0's l1: 0.315004
[27]	valid_0's l1: 0.309395
[28]	valid_0's l1: 0.304075
[29]	valid_0's l1: 0.299161
[30]	valid_0's l1: 0.294593
[31]	valid_0's l1: 0.290243
[32]	valid_0's l1: 0.286081
[33]	valid_0's l1: 0.282193
[34]	valid_0's l1: 0.278594
[

[289]	valid_0's l1: 0.202298
[290]	valid_0's l1: 0.202284
[291]	valid_0's l1: 0.202274
[292]	valid_0's l1: 0.202255
[293]	valid_0's l1: 0.20225
[294]	valid_0's l1: 0.202222
[295]	valid_0's l1: 0.202162
[296]	valid_0's l1: 0.202095
[297]	valid_0's l1: 0.202039
[298]	valid_0's l1: 0.201983
[299]	valid_0's l1: 0.201942
[300]	valid_0's l1: 0.201906
[301]	valid_0's l1: 0.201903
[302]	valid_0's l1: 0.20189
[303]	valid_0's l1: 0.201839
[304]	valid_0's l1: 0.2018
[305]	valid_0's l1: 0.20173
[306]	valid_0's l1: 0.20169
[307]	valid_0's l1: 0.201647
[308]	valid_0's l1: 0.201626
[309]	valid_0's l1: 0.201618
[310]	valid_0's l1: 0.201606
[311]	valid_0's l1: 0.20159
[312]	valid_0's l1: 0.201545
[313]	valid_0's l1: 0.201512
[314]	valid_0's l1: 0.201469
[315]	valid_0's l1: 0.201429
[316]	valid_0's l1: 0.201389
[317]	valid_0's l1: 0.201351
[318]	valid_0's l1: 0.201295
[319]	valid_0's l1: 0.201278
[320]	valid_0's l1: 0.201236
[321]	valid_0's l1: 0.201218
[322]	valid_0's l1: 0.201211
[323]	valid_0's l1: 0

[577]	valid_0's l1: 0.196194
[578]	valid_0's l1: 0.196181
[579]	valid_0's l1: 0.196159
[580]	valid_0's l1: 0.196147
[581]	valid_0's l1: 0.196141
[582]	valid_0's l1: 0.19614
[583]	valid_0's l1: 0.196124
[584]	valid_0's l1: 0.196119
[585]	valid_0's l1: 0.196117
[586]	valid_0's l1: 0.196101
[587]	valid_0's l1: 0.196088
[588]	valid_0's l1: 0.196067
[589]	valid_0's l1: 0.196055
[590]	valid_0's l1: 0.196043
[591]	valid_0's l1: 0.196028
[592]	valid_0's l1: 0.196015
[593]	valid_0's l1: 0.196007
[594]	valid_0's l1: 0.195977
[595]	valid_0's l1: 0.195952
[596]	valid_0's l1: 0.195935
[597]	valid_0's l1: 0.195898
[598]	valid_0's l1: 0.195903
[599]	valid_0's l1: 0.19589
[600]	valid_0's l1: 0.195882
[601]	valid_0's l1: 0.195874
[602]	valid_0's l1: 0.19587
[603]	valid_0's l1: 0.195866
[604]	valid_0's l1: 0.195844
[605]	valid_0's l1: 0.195832
[606]	valid_0's l1: 0.195814
[607]	valid_0's l1: 0.195778
[608]	valid_0's l1: 0.195769
[609]	valid_0's l1: 0.195749
[610]	valid_0's l1: 0.195731
[611]	valid_0's l

[865]	valid_0's l1: 0.192939
[866]	valid_0's l1: 0.192931
[867]	valid_0's l1: 0.19293
[868]	valid_0's l1: 0.192922
[869]	valid_0's l1: 0.19291
[870]	valid_0's l1: 0.192914
[871]	valid_0's l1: 0.192907
[872]	valid_0's l1: 0.192907
[873]	valid_0's l1: 0.192894
[874]	valid_0's l1: 0.192887
[875]	valid_0's l1: 0.192887
[876]	valid_0's l1: 0.192883
[877]	valid_0's l1: 0.192883
[878]	valid_0's l1: 0.192877
[879]	valid_0's l1: 0.192865
[880]	valid_0's l1: 0.192858
[881]	valid_0's l1: 0.192835
[882]	valid_0's l1: 0.192829
[883]	valid_0's l1: 0.192823
[884]	valid_0's l1: 0.192805
[885]	valid_0's l1: 0.19281
[886]	valid_0's l1: 0.192789
[887]	valid_0's l1: 0.192779
[888]	valid_0's l1: 0.192776
[889]	valid_0's l1: 0.192768
[890]	valid_0's l1: 0.192752
[891]	valid_0's l1: 0.192737
[892]	valid_0's l1: 0.192727
[893]	valid_0's l1: 0.192713
[894]	valid_0's l1: 0.19271
[895]	valid_0's l1: 0.192693
[896]	valid_0's l1: 0.19269
[897]	valid_0's l1: 0.192676
[898]	valid_0's l1: 0.192663
[899]	valid_0's l1:

...

[6700]	valid_0's l1: 0.182252
[6701]	valid_0's l1: 0.182254
[6702]	valid_0's l1: 0.182254
[6703]	valid_0's l1: 0.182249
[6704]	valid_0's l1: 0.182248
[6705]	valid_0's l1: 0.182251
[6706]	valid_0's l1: 0.182251
[6707]	valid_0's l1: 0.182251
[6708]	valid_0's l1: 0.182251
[6709]	valid_0's l1: 0.18225
[6710]	valid_0's l1: 0.182248
[6711]	valid_0's l1: 0.182251
[6712]	valid_0's l1: 0.182249
[6713]	valid_0's l1: 0.182253
[6714]	valid_0's l1: 0.182255
[6715]	valid_0's l1: 0.18225
[6716]	valid_0's l1: 0.182251
[6717]	valid_0's l1: 0.182254
[6718]	valid_0's l1: 0.182253
[6719]	valid_0's l1: 0.182256
[6720]	valid_0's l1: 0.182255
[6721]	valid_0's l1: 0.182255
[6722]	valid_0's l1: 0.182254
[6723]	valid_0's l1: 0.182255
[6724]	valid_0's l1: 0.182251
[6725]	valid_0's l1: 0.182253
[6726]	valid_0's l1: 0.182257
[6727]	valid_0's l1: 0.182254
[6728]	valid_0's l1: 0.182255
[6729]	valid_0's l1: 0.182257
[6730]	valid_0's l1: 0.182256
[6731]	valid_0's l1: 0.182254
[6732]	valid_0's l1: 0.182251
[6733]	valid

[6975]	valid_0's l1: 0.182186
[6976]	valid_0's l1: 0.182183
[6977]	valid_0's l1: 0.182181
[6978]	valid_0's l1: 0.18218
[6979]	valid_0's l1: 0.182177
[6980]	valid_0's l1: 0.182178
[6981]	valid_0's l1: 0.182173
[6982]	valid_0's l1: 0.182173
[6983]	valid_0's l1: 0.182175
[6984]	valid_0's l1: 0.182177
[6985]	valid_0's l1: 0.182179
[6986]	valid_0's l1: 0.182176
[6987]	valid_0's l1: 0.182179
[6988]	valid_0's l1: 0.18218
[6989]	valid_0's l1: 0.182185
[6990]	valid_0's l1: 0.182181
[6991]	valid_0's l1: 0.182177
[6992]	valid_0's l1: 0.182175
[6993]	valid_0's l1: 0.182175
[6994]	valid_0's l1: 0.182173
[6995]	valid_0's l1: 0.182172
[6996]	valid_0's l1: 0.182171
[6997]	valid_0's l1: 0.182166
[6998]	valid_0's l1: 0.182166
[6999]	valid_0's l1: 0.182168
[7000]	valid_0's l1: 0.182168
[7001]	valid_0's l1: 0.182162
[7002]	valid_0's l1: 0.182167
[7003]	valid_0's l1: 0.182165
[7004]	valid_0's l1: 0.182163
[7005]	valid_0's l1: 0.182163
[7006]	valid_0's l1: 0.182165
[7007]	valid_0's l1: 0.182163
[7008]	valid

[7257]	valid_0's l1: 0.182027
[7258]	valid_0's l1: 0.18203
[7259]	valid_0's l1: 0.18203
[7260]	valid_0's l1: 0.182031
[7261]	valid_0's l1: 0.18203
[7262]	valid_0's l1: 0.18203
[7263]	valid_0's l1: 0.18203
[7264]	valid_0's l1: 0.182029
[7265]	valid_0's l1: 0.182028
[7266]	valid_0's l1: 0.182029
[7267]	valid_0's l1: 0.182028
[7268]	valid_0's l1: 0.182031
[7269]	valid_0's l1: 0.182032
[7270]	valid_0's l1: 0.182033
[7271]	valid_0's l1: 0.182032
[7272]	valid_0's l1: 0.182033
[7273]	valid_0's l1: 0.182032
[7274]	valid_0's l1: 0.182032
[7275]	valid_0's l1: 0.182029
[7276]	valid_0's l1: 0.182033
[7277]	valid_0's l1: 0.182033
[7278]	valid_0's l1: 0.182035
[7279]	valid_0's l1: 0.182035
[7280]	valid_0's l1: 0.182036
[7281]	valid_0's l1: 0.182034
[7282]	valid_0's l1: 0.182037
[7283]	valid_0's l1: 0.18204
[7284]	valid_0's l1: 0.182041
[7285]	valid_0's l1: 0.182039
[7286]	valid_0's l1: 0.182041
[7287]	valid_0's l1: 0.182041
[7288]	valid_0's l1: 0.182041
[7289]	valid_0's l1: 0.182043
[7290]	valid_0's

[7538]	valid_0's l1: 0.18197
[7539]	valid_0's l1: 0.181966
[7540]	valid_0's l1: 0.181965
[7541]	valid_0's l1: 0.181959
[7542]	valid_0's l1: 0.181959
[7543]	valid_0's l1: 0.181961
[7544]	valid_0's l1: 0.181961
[7545]	valid_0's l1: 0.18196
[7546]	valid_0's l1: 0.181959
[7547]	valid_0's l1: 0.181961
[7548]	valid_0's l1: 0.181959
[7549]	valid_0's l1: 0.181961
[7550]	valid_0's l1: 0.181958
[7551]	valid_0's l1: 0.181959
[7552]	valid_0's l1: 0.181959
[7553]	valid_0's l1: 0.181961
[7554]	valid_0's l1: 0.181966
[7555]	valid_0's l1: 0.181967
[7556]	valid_0's l1: 0.181965
[7557]	valid_0's l1: 0.181962
[7558]	valid_0's l1: 0.181961
[7559]	valid_0's l1: 0.181958
[7560]	valid_0's l1: 0.181959
[7561]	valid_0's l1: 0.181959
[7562]	valid_0's l1: 0.181959
[7563]	valid_0's l1: 0.181956
[7564]	valid_0's l1: 0.181954
[7565]	valid_0's l1: 0.181955
[7566]	valid_0's l1: 0.181948
[7567]	valid_0's l1: 0.181949
[7568]	valid_0's l1: 0.181951
[7569]	valid_0's l1: 0.18195
[7570]	valid_0's l1: 0.181949
[7571]	valid_

[7815]	valid_0's l1: 0.181916
[7816]	valid_0's l1: 0.181916
[7817]	valid_0's l1: 0.181918
[7818]	valid_0's l1: 0.181917
[7819]	valid_0's l1: 0.181917
[7820]	valid_0's l1: 0.181915
[7821]	valid_0's l1: 0.181915
[7822]	valid_0's l1: 0.181916
[7823]	valid_0's l1: 0.181921
[7824]	valid_0's l1: 0.181923
[7825]	valid_0's l1: 0.181916
[7826]	valid_0's l1: 0.181914
[7827]	valid_0's l1: 0.181912
[7828]	valid_0's l1: 0.181912
[7829]	valid_0's l1: 0.181914
[7830]	valid_0's l1: 0.181913
[7831]	valid_0's l1: 0.181912
[7832]	valid_0's l1: 0.181914
[7833]	valid_0's l1: 0.181915
[7834]	valid_0's l1: 0.181915
[7835]	valid_0's l1: 0.181913
[7836]	valid_0's l1: 0.181915
[7837]	valid_0's l1: 0.181918
[7838]	valid_0's l1: 0.181918
[7839]	valid_0's l1: 0.181918
[7840]	valid_0's l1: 0.181917
[7841]	valid_0's l1: 0.181917
[7842]	valid_0's l1: 0.181921
[7843]	valid_0's l1: 0.18192
[7844]	valid_0's l1: 0.181924
[7845]	valid_0's l1: 0.181925
[7846]	valid_0's l1: 0.181927
[7847]	valid_0's l1: 0.181925
[7848]	vali

In [15]:
Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

454470.8942620657

In [13]:
Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

453729.9011332974

In [ ]:
# eran 450k antes.........

In [19]:
# Analisis de error...

In [49]:
df = X_val.copy()

df['precio'] = Y_val
df['precio_predicted'] = Y_pred
df['error'] = df.apply(lambda x: abs(x['precio']-x['precio_predicted']), axis=1).astype(int)

In [35]:
for columna in df.columns:
    print(columna)

antiguedad
habitaciones
garages
banos
metroscubiertos
metrostotales
idzona
lat
lng
gimnasio
usosmultiples
piscina
escuelascercanas
centroscomercialescercanos
metroscubiertos_alt
metrostotales_alt
metrostotales_confiables_alt
tipodepropiedad_le
es_Distrito Federal
es_Edo. de México
provincia_0_binary
provincia_1_binary
provincia_2_binary
provincia_3_binary
provincia_4_binary
provincia_5_binary
provincia_6_binary
ciudad_le
ciudad_0_binary
ciudad_1_binary
ciudad_2_binary
ciudad_3_binary
ciudad_4_binary
ciudad_5_binary
ciudad_6_binary
ciudad_7_binary
ciudad_top50_1_ohe
ciudad_top50_2_ohe
ciudad_top50_3_ohe
ciudad_top50_4_ohe
ciudad_top50_5_ohe
ciudad_top50_6_ohe
ciudad_top50_7_ohe
ciudad_top50_8_ohe
ciudad_top50_9_ohe
ciudad_top50_10_ohe
ciudad_top50_11_ohe
ciudad_top50_12_ohe
ciudad_top50_13_ohe
ciudad_top50_14_ohe
ciudad_top50_15_ohe
ciudad_top50_16_ohe
ciudad_top50_17_ohe
ciudad_top50_18_ohe
ciudad_top50_19_ohe
ciudad_top50_20_ohe
ciudad_top50_21_ohe
ciudad_top50_22_ohe
ciudad_top50_23_

In [53]:
df.groupby('aniomes')['error'].count()

aniomes
201201.0    154
201202.0    125
201203.0    112
201204.0    134
201205.0    177
201206.0    158
201207.0    205
201208.0    384
201209.0    251
201210.0    299
201211.0    155
201212.0    132
201301.0    189
201302.0    145
201303.0    168
201304.0    162
201305.0    213
201306.0    194
201307.0    280
201308.0    226
201309.0    307
201310.0    399
201311.0    400
201312.0    317
201401.0    251
201402.0    263
201403.0    317
201404.0    285
201405.0    314
201406.0    299
201407.0    295
201408.0    342
201409.0    385
201410.0    394
201411.0    468
201412.0    473
201501.0    463
201502.0    369
201503.0    383
201504.0    374
201505.0    367
201506.0    370
201507.0    391
201508.0    450
201509.0    508
201510.0    513
201511.0    491
201512.0    417
201601.0    545
201602.0    463
201603.0    514
201604.0    656
201605.0    533
201606.0    700
201607.0    609
201608.0    668
201609.0    639
201610.0    680
201611.0    622
Name: error, dtype: int64

In [55]:
df.groupby('es_Distrito Federal')['error'].agg('mean')

es_Distrito Federal
0    384327.814918
1    700878.962459
Name: error, dtype: float64

In [14]:
# respuesta para kaggle

In [16]:
ids = test_selected.index.values
X_test = test_selected.values

In [17]:
test_predict = reg.predict(X_test)

f = np.vectorize(math.exp)
test_predict = f(test_predict)

In [18]:
escribir_respuesta(ids, test_predict)

In [15]:
# en Y_pred estan los resultados predecidos y en Y_val los verdaderos. Analizaremos en donde nos
# estamos equivocando:

In [52]:
df = X_val.copy()

In [53]:
df['precio'] = Y_val

In [54]:
df['precio_predicted'] = Y_pred

In [56]:
df['error'] = df.apply(lambda x: abs(x['precio'] - x['precio_predicted']), axis=1)

In [61]:
df.loc[df['error'] < 100000]

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio',
       ...
       'antiguedad_binning_2_3_ohe2', 'antiguedad_binning_2_4_ohe2',
       'usd_precio_promedio_mensual', 'usd_subio', 'volcan_cerca',
       'volcanes_cerca', 'idzona_mean_price', 'precio', 'precio_predicted',
       'error'],
      dtype='object', length=116)

In [72]:
df.groupby('dic2016')['error'].mean()

dic2016
0    469154.720787
1    558605.942472
Name: error, dtype: float64

In [71]:
df['dic2016'].value_counts()

0    42246
1     5754
Name: dic2016, dtype: int64

In [64]:
df.groupby('mes')['error'].mean()

mes
1     457792.162680
2     463169.944441
3     456001.596693
4     492672.976551
5     450777.269360
6     470895.831920
7     465610.862803
8     461132.009234
9     484955.481680
10    485033.971211
11    472161.605521
12    525364.342841
Name: error, dtype: float64

### Modelo: Regresion lineal

In [ ]:
# ...

### Modelo: Regresion logistica

In [5]:
# ...

### Modelo: SVM

In [6]:
# ...

### Modelo: Decision Tree

In [7]:
# ...

### Modelo: RandomForest

In [4]:
from sklearn.ensemble import RandomForestRegressor

In [5]:
df = pre_processing.load_featured_appended_dataset()

In [10]:
features = feature_generation.get_features()

for feature in features:
    todas = []
    for opcion in features[feature]:
        valores = features[feature][opcion]
        for valor in valores:
            if (valor not in todas):
                todas.append(valor)
    features[feature]['todas'] = todas

In [11]:
# Como sabemos, este modelo no admite nulos, por lo que utilizaremos alguna tecnica de imputacion de los mismos
# para poder correrlo. Primero nos quedamos con las features mas importantes...
df = df[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
                        'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
                        'centroscomercialescercanos']\
                       +features["metros"][1]\
                       +features["tipodepropiedad"][0]\
                       +features["provincia"][6]\
                       +features["ciudad"]['todas']\
                       +features["fecha"][18]\
                       +features["descripcion"][0]\
                       +features["metricas"][2]\
                       +features["habitaciones"][0]\
                       +features["antiguedad"][1]\
                       +features["extras"][2]\
                       +features["volcanes"][0]\
                       +features["idzona"][5]\
                       +["precio"]]

In [ ]:
# Ahora utilizaremos una funcion definida en pre_processing para, utilizando otros regresores,
# predecir el valor que tienen las columnas con valores nulos.

df = pre_processing.fill_nans_lgb(df)

In [10]:
X = train.drop('precio', axis=1).values
Y = train['precio'].values
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

In [11]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = seed, verbose=2, max_depth=10) 
regressor.fit(X_train, Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  5.1min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=7, verbose=2,
                      warm_start=False)

In [12]:
from sklearn import metrics

In [15]:
y_pred = regressor.predict(X_val)
print('MAE: ', int(metrics.mean_absolute_error(Y_val, y_pred)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


MAE:  688548


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished


In [16]:
y_pred2 = regressor.predict(X_train)
print('MAE: ', int(metrics.mean_absolute_error(Y_train, y_pred2)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


MAE:  663067


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished


In [20]:
names = train.columns.to_list()
print(sorted(zip(map(lambda x: round(x, 4), regressor.feature_importances_), names), reverse=True))

[(0.4881, 'metroscubiertos'), (0.2571, 'ciudad_le'), (0.0352, 'ciudad_muycara'), (0.0317, 'banos'), (0.0158, 'tipodepropiedad_1_pol'), (0.0141, 'dia'), (0.0129, 'precio_promedio_metrocubierto_mes'), (0.0125, 'antiguedad'), (0.0113, 'garages'), (0.0105, 'servicio'), (0.0096, 'es_Veracruz'), (0.0093, 'metroscubiertos_mean'), (0.009, 'precio'), (0.0085, 'intercept_pol'), (0.0069, 'tipodepropiedad_2_pol'), (0.0065, 'tipodepropiedad_0_pol'), (0.005, 'habitaciones'), (0.0042, 'aniomes'), (0.0033, 'tipodepropiedad_3_pol'), (0.0031, 'ciudad_barata'), (0.0025, 'es_apart'), (0.0024, 'tipodepropiedad_4_pol'), (0.002, 'tipodepropiedad_le'), (0.002, 'ciudad_cara'), (0.0019, 'tipodepropiedad_8_ohe'), (0.0017, 'lujo'), (0.0017, 'aniomes_scaled'), (0.0015, 'mes'), (0.0015, 'es_casa'), (0.0014, 'tipodepropiedad_7_pol'), (0.0014, 'hab_binning_1_ohe'), (0.0013, 'provincia_10_ohe'), (0.0013, 'gimnasio'), (0.0012, 'parrilla'), (0.0011, 'piscina'), (0.0011, 'es_Distrito Federal'), (0.001, 'hab_binning_7_ohe

### Modelo: XGBoost

_Generacion del dataset de train con sus features_

In [4]:
import xgboost
from sklearn.model_selection import GridSearchCV

In [5]:
train,test = pre_processing.load_featured_datasets()

In [6]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [7]:
train_selected = feature_selection.get_selected_dataframe(train)
test_selected = feature_selection.get_selected_dataframe(test, precio=False)

In [8]:
X = train_selected.drop('precio', axis=1).values
Y = train_selected['precio'].values

In [18]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

In [28]:
parametros = {
    'max_depth':[11,12,13,14,15],
    'n_estimators':[100,110,120,130,140],
    'learning_rate': [0.05,0.08,0.1,0.15,0.2,0.3],
    'subsample':[0.5,0.8,0.9,0.7],
    'min_child_weight':[5,10,15,20,30]
}

In [16]:
reg = xgboost.XGBRegressor(max_depth=12,n_estimators=140 ,learning_rate=0.08, verbosity=2,subsample=0.9, min_child_weight=20, n_jobs=2)
reg.fit(X_train,Y_train)

[20:19:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:19:24] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=8
[20:19:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=8
[20:19:26] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=7
[20:19:27] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=8
[20:19:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=8
[20:19:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=9
[20:19:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes

[20:21:06] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1252 extra nodes, 0 pruned nodes, max_depth=12
[20:21:07] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1182 extra nodes, 0 pruned nodes, max_depth=12
[20:21:09] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 812 extra nodes, 0 pruned nodes, max_depth=12
[20:21:10] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1424 extra nodes, 0 pruned nodes, max_depth=12
[20:21:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1128 extra nodes, 0 pruned nodes, max_depth=12
[20:21:13] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1518 extra nodes, 0 pruned nodes, max_depth=12
[20:21:15] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1134 extra nodes, 0 pruned nodes, max_depth=12
[20:21:16] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1

[20:22:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1054 extra nodes, 0 pruned nodes, max_depth=12
[20:22:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 478 extra nodes, 0 pruned nodes, max_depth=12
[20:22:43] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 828 extra nodes, 0 pruned nodes, max_depth=12
[20:22:46] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 550 extra nodes, 0 pruned nodes, max_depth=12
[20:22:49] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=12
[20:22:52] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 788 extra nodes, 0 pruned nodes, max_depth=12
[20:22:55] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1296 extra nodes, 0 pruned nodes, max_depth=12
[20:22:58] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roo

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=12, min_child_weight=20, missing=None, n_estimators=140,
             n_jobs=2, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=2)

_Comprobacion contra el conjunto de validacion_

In [19]:
Y_pred = reg.predict(X_val)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val)
mean_absolute_error(Y_val,Y_pred)

414650.4048153364

In [26]:
Y_pred = reg.predict(X_val)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val)
mean_absolute_error(Y_val,Y_pred)

473494.35926695546

In [12]:
# preparamos el csv de respuesta para kaggle

In [13]:
ids = test_selected.index.values
X_test = test_selected.values

In [14]:
test_predict = reg.predict(X_test)

f = np.vectorize(math.exp)
test_predict = f(test_predict)

In [15]:
escribir_respuesta(ids, test_predict)

### Modelo: CatBoost

In [ ]:
#...

### Modelo: LightGBM

In [73]:
import lightgbm as lgb

In [74]:
train,test = load_featured_datasets()

In [75]:
features = feature_generation.get_features()

In [76]:
best_features = feature_selection.get_best_features_per_category()

In [98]:
features['metros']

{0: ['metroscubiertos_alt', 'metrostotales_alt'],
 1: ['metroscubiertos_alt',
  'metrostotales_alt',
  'metrostotales_confiables_alt'],
 2: ['metroscubiertos_i1', 'metrostotales_i1'],
 3: ['metroscubiertos_i1', 'metrostotales_i1', 'metrostotales_confiables_alt'],
 4: ['metroscubiertos_alt', 'metrostotales_i2'],
 5: ['metroscubiertos_alt',
  'metrostotales_i2',
  'metrostotales_confiables_alt']}

In [77]:
best_features

[('metros', 1),
 ('tipodepropiedad', 0),
 ('provincia', 6),
 ('ciudad', 2),
 ('fecha', 4),
 ('descripcion', 0),
 ('metricas', 2),
 ('habitaciones', 0),
 ('antiguedad', 1),
 ('extras', 2),
 ('volcanes', 0),
 ('idzona', 0)]

In [114]:
train_selected = train[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
                        'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
                        'centroscomercialescercanos']\
                       +features["metros"][1]\
                       +features["tipodepropiedad"][0]\
                       +features["provincia"][6]\
                       +features["ciudad"][2]\
                       +features["fecha"][4]\
                       +features["descripcion"][0]\
                       +features["metricas"][2]\
                       +features["habitaciones"][0]\
                       +features["antiguedad"][1]\
                       +features["extras"][2]\
                       +features["volcanes"][0]\
                       +features["idzona"][0]\
                       +["precio"]]

In [115]:
X = train_selected.drop('precio', axis=1).values
Y = train_selected['precio'].values

In [116]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [117]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14, 
    'learning_rate': 0.2,
    'verbose': 0, 
    'early_stopping_round': 50}
n_estimators=10000

In [118]:
d_train = lgb.Dataset(X_train, label=Y_train)
d_valid = lgb.Dataset(X_val, label=Y_val)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

[1]	valid_0's l1: 1.37848e+06
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 1.20026e+06
[3]	valid_0's l1: 1.06646e+06
[4]	valid_0's l1: 964527
[5]	valid_0's l1: 886962
[6]	valid_0's l1: 825819
[7]	valid_0's l1: 780005
[8]	valid_0's l1: 742121
[9]	valid_0's l1: 713473
[10]	valid_0's l1: 687841
[11]	valid_0's l1: 669070
[12]	valid_0's l1: 654806
[13]	valid_0's l1: 642552
[14]	valid_0's l1: 632085
[15]	valid_0's l1: 623036
[16]	valid_0's l1: 616622
[17]	valid_0's l1: 611148
[18]	valid_0's l1: 606440
[19]	valid_0's l1: 600293
[20]	valid_0's l1: 596400
[21]	valid_0's l1: 592895
[22]	valid_0's l1: 589279
[23]	valid_0's l1: 586698
[24]	valid_0's l1: 583480
[25]	valid_0's l1: 580945
[26]	valid_0's l1: 579340
[27]	valid_0's l1: 576913
[28]	valid_0's l1: 575237
[29]	valid_0's l1: 572096
[30]	valid_0's l1: 570468
[31]	valid_0's l1: 568829
[32]	valid_0's l1: 567254
[33]	valid_0's l1: 566310
[34]	valid_0's l1: 565160
[35]	valid_0's l1: 563258
[36]	valid_0's l1: 5621

[313]	valid_0's l1: 513172
[314]	valid_0's l1: 513125
[315]	valid_0's l1: 513092
[316]	valid_0's l1: 513154
[317]	valid_0's l1: 513044
[318]	valid_0's l1: 513021
[319]	valid_0's l1: 512951
[320]	valid_0's l1: 512956
[321]	valid_0's l1: 512850
[322]	valid_0's l1: 512802
[323]	valid_0's l1: 512664
[324]	valid_0's l1: 512586
[325]	valid_0's l1: 512481
[326]	valid_0's l1: 512538
[327]	valid_0's l1: 512486
[328]	valid_0's l1: 512340
[329]	valid_0's l1: 512216
[330]	valid_0's l1: 512170
[331]	valid_0's l1: 512064
[332]	valid_0's l1: 511962
[333]	valid_0's l1: 511925
[334]	valid_0's l1: 511863
[335]	valid_0's l1: 511841
[336]	valid_0's l1: 511708
[337]	valid_0's l1: 511644
[338]	valid_0's l1: 511596
[339]	valid_0's l1: 511572
[340]	valid_0's l1: 511551
[341]	valid_0's l1: 511514
[342]	valid_0's l1: 511462
[343]	valid_0's l1: 511476
[344]	valid_0's l1: 511431
[345]	valid_0's l1: 511429
[346]	valid_0's l1: 511443
[347]	valid_0's l1: 511366
[348]	valid_0's l1: 511306
[349]	valid_0's l1: 511244
[

[623]	valid_0's l1: 502016
[624]	valid_0's l1: 501982
[625]	valid_0's l1: 501985
[626]	valid_0's l1: 501975
[627]	valid_0's l1: 502002
[628]	valid_0's l1: 501962
[629]	valid_0's l1: 501937
[630]	valid_0's l1: 501924
[631]	valid_0's l1: 501931
[632]	valid_0's l1: 501862
[633]	valid_0's l1: 501834
[634]	valid_0's l1: 501766
[635]	valid_0's l1: 501745
[636]	valid_0's l1: 501754
[637]	valid_0's l1: 501773
[638]	valid_0's l1: 501770
[639]	valid_0's l1: 501770
[640]	valid_0's l1: 501752
[641]	valid_0's l1: 501748
[642]	valid_0's l1: 501765
[643]	valid_0's l1: 501754
[644]	valid_0's l1: 501685
[645]	valid_0's l1: 501644
[646]	valid_0's l1: 501590
[647]	valid_0's l1: 501591
[648]	valid_0's l1: 501581
[649]	valid_0's l1: 501612
[650]	valid_0's l1: 501543
[651]	valid_0's l1: 501540
[652]	valid_0's l1: 501484
[653]	valid_0's l1: 501469
[654]	valid_0's l1: 501456
[655]	valid_0's l1: 501450
[656]	valid_0's l1: 501455
[657]	valid_0's l1: 501469
[658]	valid_0's l1: 501405
[659]	valid_0's l1: 501433
[

[933]	valid_0's l1: 497524
[934]	valid_0's l1: 497537
[935]	valid_0's l1: 497519
[936]	valid_0's l1: 497482
[937]	valid_0's l1: 497475
[938]	valid_0's l1: 497485
[939]	valid_0's l1: 497456
[940]	valid_0's l1: 497468
[941]	valid_0's l1: 497450
[942]	valid_0's l1: 497469
[943]	valid_0's l1: 497474
[944]	valid_0's l1: 497489
[945]	valid_0's l1: 497451
[946]	valid_0's l1: 497457
[947]	valid_0's l1: 497434
[948]	valid_0's l1: 497449
[949]	valid_0's l1: 497436
[950]	valid_0's l1: 497355
[951]	valid_0's l1: 497334
[952]	valid_0's l1: 497332
[953]	valid_0's l1: 497336
[954]	valid_0's l1: 497291
[955]	valid_0's l1: 497268
[956]	valid_0's l1: 497305
[957]	valid_0's l1: 497286
[958]	valid_0's l1: 497280
[959]	valid_0's l1: 497245
[960]	valid_0's l1: 497257
[961]	valid_0's l1: 497253
[962]	valid_0's l1: 497228
[963]	valid_0's l1: 497214
[964]	valid_0's l1: 497177
[965]	valid_0's l1: 497152
[966]	valid_0's l1: 497171
[967]	valid_0's l1: 497128
[968]	valid_0's l1: 497134
[969]	valid_0's l1: 497099
[

[1230]	valid_0's l1: 494993
[1231]	valid_0's l1: 494972
[1232]	valid_0's l1: 494965
[1233]	valid_0's l1: 494977
[1234]	valid_0's l1: 494987
[1235]	valid_0's l1: 495000
[1236]	valid_0's l1: 494990
[1237]	valid_0's l1: 495010
[1238]	valid_0's l1: 495000
[1239]	valid_0's l1: 495021
[1240]	valid_0's l1: 495021
[1241]	valid_0's l1: 494993
[1242]	valid_0's l1: 494989
[1243]	valid_0's l1: 494952
[1244]	valid_0's l1: 494950
[1245]	valid_0's l1: 494952
[1246]	valid_0's l1: 494930
[1247]	valid_0's l1: 494900
[1248]	valid_0's l1: 494853
[1249]	valid_0's l1: 494854
[1250]	valid_0's l1: 494837
[1251]	valid_0's l1: 494848
[1252]	valid_0's l1: 494857
[1253]	valid_0's l1: 494868
[1254]	valid_0's l1: 494890
[1255]	valid_0's l1: 494880
[1256]	valid_0's l1: 494826
[1257]	valid_0's l1: 494789
[1258]	valid_0's l1: 494821
[1259]	valid_0's l1: 494785
[1260]	valid_0's l1: 494806
[1261]	valid_0's l1: 494812
[1262]	valid_0's l1: 494834
[1263]	valid_0's l1: 494836
[1264]	valid_0's l1: 494844
[1265]	valid_0's l1:

[1537]	valid_0's l1: 493350
[1538]	valid_0's l1: 493348
[1539]	valid_0's l1: 493358
[1540]	valid_0's l1: 493358
[1541]	valid_0's l1: 493375
[1542]	valid_0's l1: 493373
[1543]	valid_0's l1: 493371
[1544]	valid_0's l1: 493375
[1545]	valid_0's l1: 493378
[1546]	valid_0's l1: 493341
[1547]	valid_0's l1: 493364
[1548]	valid_0's l1: 493344
[1549]	valid_0's l1: 493331
[1550]	valid_0's l1: 493295
[1551]	valid_0's l1: 493269
[1552]	valid_0's l1: 493284
[1553]	valid_0's l1: 493272
[1554]	valid_0's l1: 493272
[1555]	valid_0's l1: 493305
[1556]	valid_0's l1: 493310
[1557]	valid_0's l1: 493321
[1558]	valid_0's l1: 493312
[1559]	valid_0's l1: 493321
[1560]	valid_0's l1: 493323
[1561]	valid_0's l1: 493287
[1562]	valid_0's l1: 493266
[1563]	valid_0's l1: 493243
[1564]	valid_0's l1: 493238
[1565]	valid_0's l1: 493239
[1566]	valid_0's l1: 493222
[1567]	valid_0's l1: 493172
[1568]	valid_0's l1: 493163
[1569]	valid_0's l1: 493123
[1570]	valid_0's l1: 493062
[1571]	valid_0's l1: 493061
[1572]	valid_0's l1:

[1832]	valid_0's l1: 491687
[1833]	valid_0's l1: 491692
[1834]	valid_0's l1: 491694
[1835]	valid_0's l1: 491679
[1836]	valid_0's l1: 491698
[1837]	valid_0's l1: 491671
[1838]	valid_0's l1: 491648
[1839]	valid_0's l1: 491662
[1840]	valid_0's l1: 491663
[1841]	valid_0's l1: 491659
[1842]	valid_0's l1: 491655
[1843]	valid_0's l1: 491636
[1844]	valid_0's l1: 491618
[1845]	valid_0's l1: 491602
[1846]	valid_0's l1: 491612
[1847]	valid_0's l1: 491623
[1848]	valid_0's l1: 491601
[1849]	valid_0's l1: 491599
[1850]	valid_0's l1: 491595
[1851]	valid_0's l1: 491606
[1852]	valid_0's l1: 491590
[1853]	valid_0's l1: 491588
[1854]	valid_0's l1: 491579
[1855]	valid_0's l1: 491599
[1856]	valid_0's l1: 491572
[1857]	valid_0's l1: 491566
[1858]	valid_0's l1: 491548
[1859]	valid_0's l1: 491553
[1860]	valid_0's l1: 491549
[1861]	valid_0's l1: 491544
[1862]	valid_0's l1: 491554
[1863]	valid_0's l1: 491565
[1864]	valid_0's l1: 491558
[1865]	valid_0's l1: 491568
[1866]	valid_0's l1: 491575
[1867]	valid_0's l1:

In [22]:
Y_pred = reg.predict(X_val)
mean_absolute_error(Y_val,Y_pred)

487268.28239609225

In [331]:
# preparamos el csv de respuesta para kaggle

In [46]:
ids = test_selected.index.values
X_test = test_selected.values

In [47]:
test_predict = reg.predict(X_test)
escribir_respuesta(ids, test_predict)

In [ ]:
# best params so far
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14, 
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 200}
n_estimators=20000

### Modelo: KNN

### Modelo: Neural Networks

In [3]:
# ...